In [ ]:
import mdsine2 as md2
import pandas as pd
import numpy as np
from pathlib import Path

# set negative binomial model inference parameters
params = md2.config.NegBinConfig(
  seed=0, burnin=100, n_samples=200,
  checkpoint=50,
  seed=0, burnin=0, n_samples=20, checkpoint=10,
  basepath=str("./negbin")
)

path = Path("md_data")
dataset = md2.dataset.parse(
  name="david-diet", 
  taxonomy=path/"taxonomy.tsv",  
  reads=path/"reads.tsv", 
  qpcr=path/"qpcr.tsv", 
  metadata=path/"metadata.tsv",
  perturbations=path/"perturbations.tsv"
)

In [ ]:
mcmc_negbin = md2.negbin.build_graph(
    params=params,
    graph_name=dataset.name,
    subjset=dataset
)

mcmc_negbin.run()

###### mcmc_negbin.run()

In [ ]:
import pathlib
from mdsine2.names import STRNAMES

a0 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A0])['mean']
a1 = md2.summary(mcmc_negbin.graph[STRNAMES.NEGBIN_A1])['mean']

basepath = pathlib.Path("./mdsine2/david")
basepath.mkdir(exist_ok=True, parents=True)

# Initialize parameters of the model (Seed = 0) burnin=50, total steps=100
params = md2.config.MDSINE2ModelConfig(
    basepath=str(basepath), 
    seed=0,
    burnin=50, 
    n_samples=100, 
    negbin_a0=a0, negbin_a1=a1, 
    checkpoint=25
)

mcmc = md2.initialize_graph(params=params, graph_name=dataset.name, subjset=dataset)
mcmc = md2.run_graph(mcmc, crash_if_error=True)

In [ ]:
import pickle

#with open('mcmc-david.pickle', 'wb') as handle:
#    pickle.dump(mcmc, handle)

#with open("mcmc-david.pickle", "rb") as f:
#    mcmc = pickle.load(f)

In [ ]:
growth = md2.summary(mcmc.graph[STRNAMES.GROWTH_VALUE])["mean"]
interactions = md2.summary(mcmc.graph[STRNAMES.INTERACTIONS_OBJ])["mean"]
n_taxa = len(growth)

perturbations = [np.ones(n_taxa), 0.5 * np.ones(n_taxa)]
starts = [8, 15]
ends = [12, 19]

dyn = md2.model.gLVDynamicsSingleClustering(growth=growth, interactions=interactions, perturbations=perturbations, perturbation_starts=starts, perturbation_ends=ends)


In [ ]:
import pandas as pd

x0 = pd.read_csv("reads.tsv", sep = "\t")["DD10"].values.reshape(-1, 1)
x = md2.integrate(dynamics=dyn, dt=.1, n_days=3, initial_conditions=x0)
pd.DataFrame(x["X"]).to_csv("x.csv")

In R, you can now run

```
trajectory <- read_csv("x.csv") %>%
  mutate(taxon = row_number()) %>%
  pivot_longer(-taxon, names_to = "time") %>%
  mutate(time = as.numeric(time))

trajectory %>%
  filter(taxon %in% as.character(1:25)) %>%
  ggplot() +
  geom_point(aes(time, value)) +
  facet_wrap(~ reorder(taxon, value)) +
  scale_y_log10()
```

to visualize the results.